# This script will extract human genes associated with a disease by pulling them from NCBI

###  The script will ask you to input a disease, such as type ll diabetes, and will return all the genes associated with that disease.
Furthermore, this script will write to a file that will contain all the coding regions from the genes you have collected from NCBI.

Again, if there is a variable that does not make sense, utilize a print statement and see what you can figure out. 

#### First we will need our imports

In [2]:
import csv
from Bio import SeqIO 
from Bio import Entrez
from collections import Counter

#### Next, the Entrez module requries an email, and its appropriate to use a working email

In [3]:
email = input('Enter your email: ')

Enter your email: tuh21796@temple.edu


#### This block doesnt run properly yet, so I need to fix this. The point is to handle an error if the user does not input a proper email, but then I need it to re-run the 'input code' to collect another email

In [11]:
term = input('What human disease would you like to find genes for? ')

What human disease would you like to find genes for? cancer


In [12]:
Entrez.email = str(email)

#### Lets break down this block a little bit 
We call the esearch function by saying Entrez.esearch, and we are saying the database we want to use is 'nucleotide', we want to retrieve 5 results, and the term (disease) we are searching for is (disease) AND humans AND CDS[feature key] (not exactly sure what this part is)
We then want to save the results as a dicitonary by using the Entrez.read function
Lastly we close the result handle.

#### so I am stuck here and I cant seem to figure out how to get this input disease term be searched on ncbi.. 

In [13]:
resulthandle = Entrez.esearch(db="nucleotide", retmax=5, term= """((("%s")) AND "Homo sapiens"[Organism] AND CDS[Feature key])""" % (term))
ereaddic = Entrez.read(resulthandle)  
resulthandle.close()

In [18]:
#print(ereaddic)

In [ ]:
fetchhandle = Entrez.efetch(db="nucleotide", id=ereaddic["IdList"], rettype="gb")
records = list(SeqIO.parse(fetchhandle, "genbank"))

####  Lets unpack this class a little bit, because this was a bit tricky to write as this was the first time I wrote a class 
As you can see I was extremely creative with the name of the class... its whats on the inside that counts.
Our first function initializes itself, and says 'self, set yourself = to seq_record'
Our next function gets a Counter of the 'seq' after we call . translate on that seq.
The final function goes through the information returned from the seq_record.features and if the type is "CDS", then grab that. We are able to use this logic because of the lamba function which can be very useful if you know how to use them. Essentially it allows us to filter all the information from the features in seq_record and return only what we want. Lastly, we say if you are a cds, extract the desired information that is stored in your zeroth position.

In [ ]:
class new_class:
    def __init__(self, seq_record):
        self.seq_record = seq_record
    
    
    def get_amino_count(self):
            return Counter(self.seq_record.seq.translate())
        
    def code_region (self):  
        cds = list(filter(lambda x : x.type == "CDS", self.seq_record.features))
        if cds:
            return cds[0].extract(self.seq_record.seq)

In [ ]:
temp_records = [new_class(rec) for rec in records]
aa_counts = [(temp_record.seq_record.id, temp_record.get_amino_count()) for temp_record in temp_records]

In [ ]:
temp_cds_reg = [new_class(c) for c in records]
cds = [(temp_cds.seq_record.id, temp_cds.code_region()) for temp_cds in temp_cds_reg]

#### The below writing block will write a csv file that contains all the counts of the amino acids for each gene that was returned.
This is a cool little dictionary writer, but at first blush it is a little overwhelming. 

So, we set the variable writer to the csv DictWriter which takes an input of our csvfile and the fieldnames. The field name section is pretty nice because you can create an order for your data (I did not, I just made sure all my characters were accounted for), and remember these letters represent amino acids.

With our for loop, we say for each counter in aa_counts (and remember there should be 5 total counters in our list because we set retmax to 5) we make the 1st position a dictionary (again, remember in python starts indexing at 0). So then we use the .update function to add 'id' to the zeroth position of our dictionary. At this point, we have a dictionary with 'id' as the key, and a dictionary as the value... this seems a little confusing but throw some print statements in this function if you need a little more visual clarity. We then write the updated dictionary to a file. 

I need to work on this a little more to error gracefully. 

In [17]:
with open('amino_acid.csv','w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["id","G","A","P","M","F","I","K","H","L","W","N","E","Y","*","T","C","S","D","V","Q","R"])
    writer.writeheader()
    for j in aa_counts:
        writer1 = dict(j[1])
        writer1.update({"id":j[0]})
        writer.writerow(writer1)

NameError: name 'aa_counts' is not defined

#### This neat little block will grab the sequences ids and the sequences from our records list and append the info to our new lists.
We will use this info later.

In [ ]:
seq_ids = [record.id for record in records] 
seqs = [record.seq for record in records]

#### This is just a quick little for loop that will collect all the coding regions in the sequences 

In [ ]:
cds_list = []
for c in cds:
    cds_list.append(c[1])


#### This fancy little trick turns your 3 lists in a tuple where the entries in the tuple are: [(seq_id_1, seq_1, cds_list_1), (seq_id_2, seq_2, cds_list_2), (etc)]
I have found this 'zip' function to come in extremely handy especially when I have to write outputs to tsv files with headers. 

In [ ]:
lines = zip(seq_ids, seqs, cds_list)

#### We are going to use this block to write a file that will contain our sequence ids, sequences, and coding regions of our sequences.
Again, I have used the enumerate function to insert a tab in between elements in this file.

In [ ]:
with open('db_data.tsv', 'w') as out:
    for i, line in enumerate(lines):
        out.write("{}\t{}\t{}\t{}\n".format(i+1, line[0], line[1], line[2]))

#### This next block of code will write a fasta file for all the coding regions of the genes returned from the search
    First we have to add a '>' to the beginning of each element in the list of sequence ids which will allow us to visualize the information exactly like a fasta file. Each element in the new list now looks like this: '>NM_001321858.1'. The new list is called fa_seq_ids, and this little trick comes in very handy. We then return to our beloved zip function, and finally write align_me in fasta format that is ready to be put into an alignment viewer.

In [ ]:
fa_seq_ids = ['>' + i for i in seq_ids] 
lines = zip (fa_seq_ids, cds_list)

In [ ]:
with open('align_me.fa', 'w') as out:
    for line in lines:
        out.write("{}\t{}\n".format(line[0], line[1]))